In [1]:
from urdfenvs.robots.generic_urdf import GenericUrdfReacher
from urdfenvs.sensors.full_sensor import FullSensor
from urdfenvs.urdf_common import UrdfEnv
from urdfenvs.urdf_common.reward import Reward

import numpy as np
import os

pybullet build time: May 20 2022 19:45:31


In [2]:
class DistanceReward(Reward):
    def calculate_reward(self, observation: dict) -> float:
        # goal = observation['robot_0']['FullSensor']['goals'][1]['position']
        # position = observation['robot_0']['joint_state']['position']
        # reward = 1.0/np.linalg.norm(goal-position)
        # reward = np.linalg.norm(goal - position)
        # print(f"👁️: {observation}")
        # endeffector_position_raw = observation['robot_0']['joint_state']['position'][:3]
        # print(f"🦾 End-effector Position: {endeffector_position_raw}")
        obstacle_position = observation["robot_0"]["FullSensor"]["obstacles"][1]["position"]
        # print(f'⬛ Obstacle Position: {obstacle_position}')
        target_position = np.array([0.3, -0.4, 0.0])
        reward = 1 / np.linalg.norm(obstacle_position - target_position)
        # reward = 0.0
        # print(f'🏆 Reward is: {reward}')
        return reward

In [3]:
from mpscenes.obstacles.box_obstacle import BoxObstacle


movable_obstacle_dict = {
    'type': 'box',
    'geometry': {
        'position' : [0.5, 0.5, 0.0],
        'width': 0.04,
        'height': 0.04,
        'length': 0.1,
    },
    'movable': True,
    'high': {
         'position' : [5.0, 5.0, 1.0],
        'width': 0.35,
        'height': 0.35,
        'length': 0.35,
    },
    'low': {
        'position' : [0.0, 0.0, 0.5],
        'width': 0.2,
        'height': 0.2,
        'length': 0.2,
    }
}
movable_obstacle = BoxObstacle(name="movable_box", content_dict=movable_obstacle_dict)

In [4]:
robots = [
    GenericUrdfReacher(urdf="panda_with_gripper.urdf", mode="vel"),
]

render: bool = False

env = UrdfEnv(render=render, robots=robots)

In [5]:
# env.add_goal(goal1)
# env.add_goal(dynamicGoal)

sensor = FullSensor(['position'], ['position', 'size'], variance=0.0)
env.add_sensor(sensor, [0])
env.add_obstacle(movable_obstacle)
env.set_reward_calculator(DistanceReward())
env.set_spaces()
# defaultAction = np.array([0.5, -0.0, 0.0])
# pos0 = np.array([0.0, 0.1, 0.0])
# vel0 = np.array([1.0, 0.0, 0.0])

ob, *_ = env.reset(
    # pos=pos0, vel=vel0
    )
# env.shuffle_goals()
print(f"Initial observation : {ob}")


/home/behradx/anaconda3/envs/SB3/lib/python3.9/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float64
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Initial observation : {'robot_0': {'joint_state': {'position': array([ 0.    ,  0.    ,  0.    , -1.501 ,  0.    ,  1.8675,  0.    ,
        0.02  ,  0.02  ]), 'velocity': array([0., 0., 0., 0., 0., 0., 0., 0., 0.])}, 'FullSensor': {'obstacles': {1: {'position': array([0.5, 0.5, 0. ], dtype=float32), 'size': array([0.1 , 0.04, 0.04], dtype=float32)}}}}}


In [6]:
print(f"🦾 End-effector Position: {ob['robot_0']['joint_state']['position']}")

🦾 End-effector Position: [ 0.      0.      0.     -1.501   0.      1.8675  0.      0.02    0.02  ]


In [7]:
# print(f'⬛ Obstacle Position: {ob["robot_0"]["FullSensor"]["obstacles"][1]["position"]}')

In [8]:
# endeffector_position = ob['robot_0']['joint_state']['position'][:3]
# print(f"🦾 End-effector Position: {endeffector_position}")
# obstacle_position = ob["robot_0"]["FullSensor"]["obstacles"][1]["position"]
# print(f'⬛ Obstacle Position: {obstacle_position}')

In [9]:
# env.reset()
# while True:
#     action = env.action_space.sample()
#     obs, reward, done, truncated, info = env.step(action)


In [10]:
# Stable baselines 3
from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise

from gymnasium.wrappers import FlattenObservation


In [11]:
MODEL_NAME = 'TD3-001'
MODEL_CLASS = TD3

In [12]:
models_dir = 'models/' + MODEL_NAME
logdir = 'logs'

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)

In [13]:
env = FlattenObservation(env)

In [14]:
from urdfenvs.wrappers.sb3_float32_action_wrapper import SB3Float32ActionWrapper

env = SB3Float32ActionWrapper(env)

In [15]:
from stable_baselines3.common.env_checker import check_env

check_env(env, warn=True)

/home/behradx/anaconda3/envs/SB3/lib/python3.9/site-packages/stable_baselines3/common/env_checker.py:422: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(


In [16]:
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

TIMESTEPS = 10000
model = MODEL_CLASS("MlpPolicy", env, verbose=1, tensorboard_log=logdir, action_noise=action_noise, device='cpu')
model.learn(total_timesteps=TIMESTEPS, log_interval=10, tb_log_name=MODEL_NAME,
             progress_bar=True,
               )
model.save(f"{MODEL_NAME}-model")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to logs/TD3-001_4


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.1      |
|    ep_rew_mean     | 1.19     |
| time/              |          |
|    episodes        | 10       |
|    fps             | 7        |
|    time_elapsed    | 1        |
|    total_timesteps | 11       |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.05     |
|    ep_rew_mean     | 1.14     |
| time/              |          |
|    episodes        | 20       |
|    fps             | 7        |
|    time_elapsed    | 2        |
|    total_timesteps | 21       |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.03     |
|    ep_rew_mean     | 1.12     |
| time/              |          |
|    episodes        | 30       |
|    fps             | 6        |
|    time_elapsed    | 4        |
|    total_timesteps | 31       |
---------------------------------


...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 1.08     |
| time/              |          |
|    episodes        | 1640     |
|    fps             | 5        |
|    time_elapsed    | 292      |
|    total_timesteps | 1641     |
| train/             |          |
|    actor_loss      | -1.09    |
|    critic_loss     | 1.87e-05 |
|    learning_rate   | 0.001    |
|    n_updates       | 1540     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 1.08     |
| time/              |          |
|    episodes        | 1650     |
|    fps             | 5        |
|    time_elapsed    | 294      |
|    total_timesteps | 1651     |
| train/             |          |
|    actor_loss      | -1.09    |
|    critic_loss     | 1.9e-05  |
|    learning_rate   | 0.001    |
|    n_updates       | 1550     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 1.08     |
| time/              |          |
|    episodes        | 1660     |
|    fps             | 5        |
|    time_elapsed    | 296      |
|    total_timesteps | 1661     |
| train/             |          |
|    actor_loss      | -1.08    |
|    critic_loss     | 5.8e-06  |
|    learning_rate   | 0.001    |
|    n_updates       | 1560     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 1.08     |
| time/              |          |
|    episodes        | 1670     |
|    fps             | 5        |
|    time_elapsed    | 298      |
|    total_timesteps | 1671     |
| train/             |          |
|    actor_loss      | -1.08    |
|    critic_loss     | 2.87e-05 |
|    learning_rate   | 0.001    |
|    n_updates       | 1570     |
---------------------------------


KeyboardInterrupt: 

In [ ]:
env.close()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
